In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

In [9]:
df = pd.read_csv("data/processed_pancan_pca.csv")

In [10]:
# ==========================================
# PREPARE FEATURES (X) AND TARGET (y)
# ==========================================

# 1. Define Features (X)
# Drop metadata/target columns to keep only clinical features + PCA components
X = df.drop(columns=['sample', 'OS', 'OS.time'])

# 2. Define Target (y)
# Scikit-survival requires a structured array: [(Status, Time), (Status, Time), ...]
# 'Status' must be boolean (True = Event occurred/Dead, False = Censored/Alive)
y = np.array(
    list(zip(df['OS'].astype(bool), df['OS.time'])),
    dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
)

print(f"Features Shape: {X.shape}")
print(f"Target Shape: {y.shape}")

Features Shape: (10952, 3396)
Target Shape: (10952,)


In [11]:
# Split into training and testing sets (80% train, 20% test)
# We stratify by 'Status' to ensure both sets have a similar proportion of events
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y['Status']
)

print(f"Train samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

Train samples: 8761
Test samples: 2191


In [12]:
# Initialize the Cox Proportional Hazards model
# alpha > 0 adds a Ridge penalty (L2 regularization) which helps with stability 
# given the large number of PCA features.
cph = CoxPHSurvivalAnalysis(alpha=1e-4, verbose=0)

print("Training CoxPH Model...")
cph.fit(X_train, y_train)
print("Training Complete!")

Training CoxPH Model...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sksurv/linear_model/coxph.py:200: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


Training Complete!


In [13]:
# ==========================================
# EVALUATION
# ==========================================

# Predict risk scores (hazard ratios)
scores = cph.predict(X_test)

# Calculate C-INDEX
c_index = concordance_index_censored(
    y_test['Status'], 
    y_test['Survival_in_days'], 
    scores
)[0]

print(f"\n==========================================")
print(f"CoxPH C-INDEX: {c_index:.4f}")
print(f"==========================================")


CoxPH C-INDEX: 0.6633
